# Basic summary of what this capstone will, and may accomplish

* Scrape data from indeed, and return job postings
* properly format location, title, requirements
* analyze what words and requirements occur the most
* Determine the quality of each job posting (how generic does it sound, and how common are the phrases that they are using) - this is important, but needs a bit of refinement

Stretch goals
* add input to allow users to specifiy the cities to look at

* add a map to visualize how many postings there are in each city 

Current Issues
* Sponsored job posts show up in data multiple times, and location data is missing
* Summary page is not formated properly, .split() doesn't fix it

# Indeed Job Scraper

So far, it can scrape a job's

* location
* title
* company name
* salary
* summary

to-do

* responsibilities
* requirements

not required, but would be nice

* additional queries
* additional cities

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [2]:
max_results_per_city = 100
#city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder']
# titles = ['data+analyst', 'business+analyst', 'data+scientist', 'machine+learning']
city_set = ['New+York']
columns = ['city', 'job_title', 'company_name', 'location', 'summary']
sample_df = pd.DataFrame(columns = columns)

In [3]:
def parse(query,metro):
    
    url = "https://www.indeed.com/jobs?q={}&l={}&start={}"
    n = 50
    
    # Create dataframe that we'll return later
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis", "Query", "Metro"])
    
    # Scrape first n pages
    for i in range(0,n):
        html = requests.get(url.format(query, metro, i))
        time.sleep(1)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = 'None'
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = 'None'
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = 'None'
            synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis, 'Query': query, 'Metro': metro}, ignore_index=True)
    return df

In [4]:
# Parse the first URL
data1 = parse('data+analyst', 'washington+dc')

In [5]:
# Parse the second URL
data2 = parse('data+scientist', 'washington+dc')

In [6]:
# Parse the third URL
data3 = parse('data+engineer', 'washington+dc')

In [7]:
# Merge the two dataframes
data = pd.concat([data1,data2,data3])

In [8]:
# Save scraped data to a local directory
if os.path.exists("data/dc_indeed.csv"):
    os.remove("data/dc_indeed.csv")
data.to_csv('data/dc_indeed.csv', encoding='utf-8')

In [9]:
# Preview the data
data.head(10)

,Title,Location,Company,Salary,Synopsis,Query,Metro
0,Intelligence Analyst - Entry/Mid-Level,None,National Security Agency,"\n $45,972 - $73,105 a year",Entry is with a hi...,data+analyst,washington+dc
1,Data Analyst,None,Callahan & Associates,None,Hands on experienc...,data+analyst,washington+dc
2,Business Intelligence Analyst,None,Washington Metropolitan Area Transit A...,None,"Data processing, data warehousing, and databas...",data+analyst,washington+dc
3,Product Modeling Analyst,None,GEICO,None,Statistical Modeli...,data+analyst,washington+dc
4,Data Analyst,None,Chenega Corporation,None,Perform data entry...,data+analyst,washington+dc
5,Health Interventions and Claims Data Analyst,"Washington, DC 20007 (Georgetown area)",Innovation Center for Biomedical Infor...,None,We are working on applying data sc...,data+analyst,washington+dc
6,Data Analyst 1 - Petroleum Supply,"Washington, DC 20585",IMG Crown Energy Services Joint Venture,"\n $40,000 - $45,000 a year",Supporting the data validation tea...,data+analyst,washington+dc
7,Data Fellowship (Spring 2018),"Alexandria, VA",Echelon Insights,"\n $1,000 a month",Fellows with proven abilities in t...,data+analyst,washington+dc
8,MS Excel Data Analyst F32711,"Washington, DC 20260 (South West area)",SyApps LLC,None,Analyzing all types of data sets. ...,data+analyst,washington+dc
9,Data Analyst 1,"Washington, DC 20585",IMG Crown Energy Services Joint Venture,"\n $42,000 - $45,000 a year",Reviews data edits to ensure accur...,data+analyst,washington+dc
